# Setting up API
Source: NASS



In [ ]:
import pandas as pd
import requests

In [ ]:
API_KEY = '9805ABF0-1BB6-30D0-BE7F-2FA955C28C81'

base_url = 'https://quickstats.nass.usda.gov/api/api_GET/'

dfs = []

# Yield API call
for crop in ['CORN', 'WHEAT']:
    params = {
            'key' : API_KEY,
            'source_desc' : 'SURVEY',
            'sector_desc' : 'CROPS',
            'group_desc' : 'FIELD CROPS',
            'commodity_desc': crop,
            'agg_level_desc' : 'STATE',
            'freq_desc' : 'ANNUAL',
            'statisticcat_desc' : 'YIELD',
            'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
            'year__GE': '1961',
            'format' : 'JSON'}
    # Explicit class rule for Wheat
    if crop == "WHEAT":
        params["class_desc"] = "ALL CLASSES"

    r = requests.get(base_url, params=params)
    print(r.status_code)
    print(r.text[:1000])
    r.raise_for_status()

    data = r.json()["data"]
    df = pd.DataFrame(data)
    df["commodity"] = crop

    dfs.append(df)

df_yield = pd.concat(dfs, ignore_index=True)

In [ ]:
# Area Harvested API call
for crop in ['CORN', 'WHEAT']:
    params = {
            'key' : API_KEY,
            'source_desc' : 'SURVEY',
            'sector_desc' : 'CROPS',
            'group_desc' : 'FIELD CROPS',
            'commodity_desc': crop,
            'agg_level_desc' : 'STATE',
            'freq_desc' : 'ANNUAL',
            'statisticcat_desc' : 'AREA HARVESTED',
            'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
            'year__GE': '1961',
            'format' : 'JSON'}
    # Explicit class rule for Wheat
    if crop == "WHEAT":
        params["class_desc"] = "ALL CLASSES"

    r = requests.get(base_url, params=params)
    print(r.status_code)
    print(r.text[:1000])
    r.raise_for_status()

    data = r.json()["data"]
    df = pd.DataFrame(data)
    df["commodity"] = crop

    dfs.append(df)

df_area = pd.concat(dfs, ignore_index=True)

# Cleaning Yield Dataframe

In [239]:
df_y = df_yield.copy()

# Keeping BU/ACRE
df_y = df_y[df_y["unit_desc"] == "BU / ACRE"].copy()

# Keeping ALL util practice
df_y = df_y[df_y["util_practice_desc"] == "ALL UTILIZATION PRACTICES"].copy()

# Dropping state aggregates
df_y = df_y[df_y["state_alpha"] != "OT"].copy()

# Col list we keep
COLS_YIELD = ["state_alpha", "state_name", "year",
             "commodity_desc", "unit_desc", "Value",
             "reference_period_desc", 'source_desc']

# Filtering out unnecessary cols
df_y = df_y[COLS_YIELD].rename(columns={
    "commodity_desc": "crop",
    "unit_desc": "yield_unit",
    "Value": "yield"
})

# Picking max value from yearly forecasts and year
df_y = (df_y.sort_values(["state_alpha","year","crop","yield"])   # YEAR=0 will be smaller than NOV>0
          .groupby(["state_alpha","year","crop"], as_index=False)
          .tail(1)
          .reset_index(drop=True))

print(df_y.duplicated(["state_alpha","year","crop"]).sum())


0


In [240]:
df_y

,state_alpha,state_name,year,crop,yield_unit,yield,reference_period_desc,source_desc
0,AL,ALABAMA,1961,WHEAT,BU / ACRE,26,YEAR,SURVEY
1,AL,ALABAMA,1962,WHEAT,BU / ACRE,24,YEAR,SURVEY
2,AL,ALABAMA,1963,WHEAT,BU / ACRE,23.5,YEAR,SURVEY
3,AL,ALABAMA,1964,WHEAT,BU / ACRE,25,YEAR,SURVEY
4,AL,ALABAMA,1965,WHEAT,BU / ACRE,24.5,YEAR,SURVEY
...,...,...,...,...,...,...,...,...
2682,WY,WYOMING,2021,WHEAT,BU / ACRE,32,YEAR,SURVEY
2683,WY,WYOMING,2022,WHEAT,BU / ACRE,17,YEAR,SURVEY
2684,WY,WYOMING,2023,WHEAT,BU / ACRE,30,YEAR,SURVEY
2685,WY,WYOMING,2024,WHEAT,BU / ACRE,31,YEAR,SURVEY


# Cleaning Area Harvested Dataframe

In [ ]:
df_a = df_area.copy()

# Keeping ACRES
df_a = df_a[df_a[('unit_desc')] == "ACRES"].copy()

# Keeping all util practices
df_a = df_a[df_a["util_practice_desc"] == "ALL UTILIZATION PRACTICES"].copy()

# Keeping year only
df_a = df_a[df_a["reference_period_desc"] == "YEAR"].copy()

# Dropping OT
df_a = df_a[df_a["state_alpha"] != "OT"].copy()

# Filtering out unnecessary columns
COLS_AREA = ["state_alpha", "state_name", "year",
             "commodity_desc", "unit_desc", "Value",
             "reference_period_desc", 'source_desc']

df_a = df_a[COLS_AREA].rename(columns={
    "commodity_desc": "crop",
    "unit_desc": "area_unit",
    "Value": "area_harvested"
})

# Picking max value from yearly forecasts and year
df_a = (df_a.sort_values(["state_alpha", "year", "crop", 'area_harvested'])
          .groupby(["state_alpha", "year", "crop"], as_index=False)
          .tail(1)
          .reset_index(drop=True))

df_a.duplicated(["state_alpha", "year", "crop"]).sum()


# Merging Dataframes

In [244]:
merged = df_y[["state_alpha", "year", "crop","yield"]].merge(
    df_a[["state_alpha", "year", "crop","area_harvested"]],
    on=["state_alpha", "year", "crop"],
    how="inner"
)

merged

,state_alpha,year,crop,yield,area_harvested
0,AL,1961,WHEAT,26,"56,000"
1,AL,1962,WHEAT,24,"35,000"
2,AL,1963,WHEAT,23.5,"42,000"
3,AL,1964,WHEAT,25,"64,000"
4,AL,1965,WHEAT,24.5,"55,000"
...,...,...,...,...,...
2682,WY,2021,WHEAT,32,"95,000"
2683,WY,2022,WHEAT,17,"95,000"
2684,WY,2023,WHEAT,30,"90,000"
2685,WY,2024,WHEAT,31,"91,000"


# Saving Dataset

In [245]:
merged.to_parquet("data/processed/crop_annual.parquet", index=False)